# Introduction to Natural Language Processing 01
## Lab 03 - FastText
Nelson VICEL--FARAH\
Karen KASPAR\
Romain BRAND

In [1]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")

imdb_dataset

Reusing dataset imdb (/home/miolith/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### 1. (2 points) Turn the dataset into a dataset compatible with Fastext (see the Tips on using FastText section a bit lower)
On transforme le dataset en données utilisables par fasttext

In [2]:
import pandas as pd
import numpy as np
from string import punctuation


def preparing(input_df):
    df = input_df.copy()
    #change values 0 and 1 to negative and positive
    df['label'] = df['label'].replace([0, 1], ['negative', 'positive'])

    #pretreatment (applying lower case and punctuation removal) 
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].str.replace('[{}]'.format(punctuation), ' ', regex=True)
    return df

def save_to_fasttext(df, filename):
    #save the content of the dataframe into a text file with the appropriate format
    np.savetxt(filename, '__label__'+df['label']+' '+df['text'], fmt='%s')

In [3]:
# On mélange les données pour éviter les biais
train_df = pd.DataFrame(imdb_dataset['train']).sample(frac=1).reset_index()
test_df = pd.DataFrame(imdb_dataset['test']).sample(frac=1).reset_index()


save_to_fasttext(preparing(train_df), "train_df.txt")
save_to_fasttext(preparing(test_df), "test_df.txt")

In [4]:
!head -n 2 'train_df.txt'

__label__positive in 1976 i had just moved to the us from ceylon  i was 23  and had been married for a little over three years  and was beginning to come out as a lesbian  i saw this movie on an old black and white tv  with terrible reception  alone  and uninterrupted  in an awakening that seemed like an echo of the story  i was living in a small house in tucson arizona  and it was summertime    like everyone else here  i never forgot the feelings the images of this story called forth  and its residue of fragile magic  and i have treasured a hope that i would see it again someday  i ll keep checking in  i also wish that someone would make a movie of shirley verel s  the other side of venus   it also has some of the same delicacy and persistent poignancy   
__label__positive mahatma has been depicted as a man who neglected his own son in this movie  don t get me wrong i am not condemning the movie  it is such a wonderful movie and walked out of the cinema with a lump in my throat  we ne

### 2. (2 points) Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

In [5]:
import fasttext

In [6]:
default_model = fasttext.train_supervised('train_df.txt')

Read 6M words
Number of words:  75910
Number of labels: 2
Progress: 100.0% words/sec/thread: 2771728 lr:  0.000000 avg.loss:  0.385381 ETA:   0h 0m 0s


In [7]:
default_model.test("test_df.txt")

(25000, 0.87808, 0.87808)

### 3. (3 points) Use the hyperparameters search functionality of FastText and repeat step 2

In [8]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_df, shuffle=True, stratify=train_df["label"])

save_to_fasttext(preparing(train), "cooking.train")
save_to_fasttext(preparing(val), "cooking.valid")

In [9]:
model = fasttext.train_supervised(input='cooking.train', autotuneValidationFile='cooking.valid')

Progress: 100.0% Trials:    9 Best score:  0.898080 ETA:   0h 0m 0s
Training again with best arguments
Read 4M words
Number of words:  67569
Number of labels: 2
Progress: 100.0% words/sec/thread: 1183372 lr:  0.000000 avg.loss:  0.050422 ETA:   0h 0m 0s


In [10]:
model.test("test_df.txt")

(25000, 0.89284, 0.89284)

### 4. (1 points) Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ?

In [11]:
print("attribute | Default  |  Autotuned")
print("lr        |", default_model.lr, "     |", model.lr)
print("epoch     |", default_model.epoch, "       |", model.epoch)
print("dim       |", default_model.dim, "     |", model.dim)
print("wordNgrams|", default_model.wordNgrams, "       |", model.wordNgrams)
print("bucket    |", default_model.bucket, "       |", model.bucket)

attribute | Default  |  Autotuned
lr        | 0.1      | 0.08499425639667486
epoch     | 5        | 100
dim       | 100      | 92
wordNgrams| 1        | 2
bucket    | 0        | 4110692


### 5. (1 point) Using the tuned model, take at least 2 wrongly classified examples from the test set, and try explaining why the model failed.

In [12]:
# On remplace les occurences de "positive" en "1" et "negative" en "0"
test_df["prediction"] = test_df["text"].apply(lambda x: int("p" in model.predict(x)[0][0]))
test_df

,index,text,label,prediction
0,15732,"""Challenge to be Free"" was one of the first fi...",1,1
1,1791,Linda Arvidson (as Jennie) and Harry Solter (a...,0,1
2,7788,This movie was one of the worst I've ever seen...,0,0
3,10624,Low-budget murder mystery about a Public Defen...,0,0
4,14800,"Well, I've just seen Buster Keaton's film debu...",1,1
...,...,...,...,...
24995,9241,"OK, last night I saw the world premiere of Pau...",0,0
24996,7262,"If it were possible, I would have given this s...",0,0
24997,19791,I enjoyed the movie and Kellie Martins perform...,1,1
24998,12905,"Okay, sure, this movie is a bit on the hokey s...",1,1


In [13]:
failed_sample1 = test_df[(test_df["prediction"] == 1) & (test_df["label"] == 0)].iloc[2,:]["text"]
failed_sample2 = test_df[(test_df["prediction"] == 0) & (test_df["label"] == 1)].iloc[2,:]["text"]

### Cet exemple est un faux positif. L'auteur du commentaire relève les défauts du film tout en soulignant ses forces, offrant ainsi une critique plus objective du film.

In [14]:
failed_sample1

'The MTV sci-fi animated series "Æon Flux" is brought to life with Charlize Theron playing the title character, a freedom fighter who fights oppression in the walled city of Bregna, 400 hundred years into the future. For her latest mission, she has been sent to kill the city\'s leader Trevor Goodchild (Marton Csokas), but she uncovers secrets along the way.<br /><br />Aeon Flux falls under the category of good premise, mediocre execution. Interesting story yet the film was a little dull. A lot of people are saying that this is one of the worst movies of the year and that\'s not true at all. It may be a disappointing film but it\'s an average film at best. I have never seen the cartoon version of the movie so I can\'t compare the two. It\'s probably better because they have a chance to explain the story more. The film is not that confusing but it\'s easy to get lost if you\'re not familiar with the material. The acting was alright, nothing special. Charlize Theron gives a good performan

### Cet exemple est un faux négatif. La première phrase peut induire en erreur ('I tuned in to this movie because there was nothing else to watch') ainsi que d'autres expressions ('The characters do stupid things', 'mindless entertainment') même si le reste du commentaire est un éloge du film.

In [15]:
failed_sample2

'I tuned in to this movie because there was nothing else to watch. I was immediately sucked in by the characters.<br /><br />Robin Tunney is nothing less than spectacular in this film. Her portrayal of a mentally ill woman is both moving and 100% believable. Really, this sort of thing is not easy to do. She pulls it off fantastically.<br /><br />We know early on this film is going to end tragically, but you cannot take your eyes off of it. The characters do stupid things, but unlike most Hollywood movies where people do stupid things because the plot demands it, these people do stupid things because the are not right in the head - and the things they do are completely consistent with their characters.<br /><br />This is just a great example of film making IMHO. Great writing, great acting, great directing. A film for people who think film can be more than mindless entertainment.'

### 6. (Bonus point) Why is it likely that the attributes minn and maxn are at 0 after an hyperparameter search on our data?

#### minn et maxnn sont à 0 car les mots anglais sont rarement une composition d'autres mots, nul besoin donc de les diviser en sous-tokens